## Understanding how to generate reponses using Ollama LLMs

#### 1. Intuition: The "Open-Book Exam"  
Imagine you are a brilliant student (the LLM), but you have amnesia.  
You remember everything up to 2023, but nothing after that.  

- **Without RAG:**  
  If I ask you about news from today, you might panic and make something up to sound smart.  
  This is called **Hallucination**.  

- **With RAG:**  
  Before you answer, I hand you a newspaper from today.  
  You read the specific article relevant to my question, and then you answer using that information.  

**RAG is simply the process of handing the LLM the right notes before it speaks.**

#### 2. Visual-in-Words: The Architecture  
Close your eyes and visualize this flow:  

1. The **User** asks a question:  
   *"How do I reset the X-2000 router?"*  

2. The **Retriever** (The Hand) reaches into a bucket of manuals (your database), rummages around, and grabs the one specific paragraph about resetting the X-2000.  

3. The **Augmenter** (The Glue) takes the User's question and tapes that paragraph to it.  
   New Prompt:  
   *"Context: [Reset paragraph]. Question: How do I reset the X-2000 router?"*  

4. The **Generator** (The Brain / ollama) reads the combined text and generates an accurate answer.

In [1]:
import ollama

# 1. Define the question
user_question = "What is the capital of India?"

# 2. Send it to the model (Generation only)
# We are NOT providing external context yet.
response = ollama.chat(
    model='gemma3:4b',  # Enter the model that you pull on ollama
    messages=[{'role': 'user', 'content': user_question}]
)

# 3. Print the answer
print("AI Answer:", response['message']['content'])

AI Answer: The capital of India is **New Delhi**. 

It’s a planned city, officially named New Delhi, though the larger area is officially called National Capital Territory of Delhi. 😊 

Do you want to know anything more about New Delhi?


We write,<br>
`response['message']['content']`<br>
because it is in JSON format:<br>

```json
{
  "model": "phi3:3.8b",
  "created_at": "2025-12-15T09:45:00.5105742Z",
  "done": true,
  "done_reason": "stop",
  "total_duration": 11129766800,
  "load_duration": 3728855400,
  "prompt_eval_count": 16,
  "prompt_eval_duration": 765201100,
  "eval_count": 94,
  "eval_duration": 6592721000,
  "message": {
    "role": "assistant",
    "content": "The capital of India is New Delhi. It became the political center in 1956 when the country's government moved from Kolkata (then called Calcutta) and now serves as a major hub for culture, education, and politics, housing Parliament House, which holds the Indian legislative bodies - Lok Sabha (House of the People), Rajya Sabha (Council of States), along with numerous other government offices.",
    "thinking": null,
    "images": null,
    "tool_name": null,
    "tool_calls": null
  },
  "logprobs": null
}



### Different Roles

| Role      | Analogy               | Purpose                                  |
|-----------|-----------------------|------------------------------------------|
| System    | The Boss / Director   | Defines behavior, tone, and rules.        |
| User      | The Customer          | Asks the question.                       |
| Assistant | The Transcript        | Provides conversation history/context.   |


```py
messages=[
    # 1. We set the context (optional but good)
    {'role': 'system', 'content': 'You are a helpful assistant.'},

    # 2. First turn
    {'role': 'user', 'content': 'Who wrote Harry Potter?'},
    {'role': 'assistant', 'content': 'J.K. Rowling wrote it.'}, # <--- WE INSERT THIS MANUALLY OR FROM PREVIOUS RESPONSE

    # 3. Current question (The AI now reads the line above and knows 'she' = Rowling)
    {'role': 'user', 'content': 'What year was she born?'}
]

## Embedding

#### 1. Intuition: The Supermarket Aisle  
Imagine you go to a supermarket and ask a clerk: “I need ingredients for a salad.”  

- **Keyword Search (The old Way):**  
The clerk looks for items that have the word “Salad” written on them.  
They hand you “Salad Dressing” and “Salad Tongs”.  
They miss the Lettuce and Tomatoes because those items don’t have the word “Salad” printed on them.  

- **Vector Search (Embeddings – The RAG Way):**  
The clerk understands the concept of a salad.  
They know that Lettuce, Tomatoes, and Cucumbers “live” in the same conceptual aisle as “Salad”, even if the words are different.  

Embeddings turn text into a coordinate on a map.  
Words with similar meanings live close together.

#### 2. Visual-in-Words: The 3-D Map  
Imagine a giant 3-D box floating in space.  
Every sentence you can possibly speak is a tiny dot inside this box.  

- The dot for “The dog barked” is at coordinate [10, 50, 3].  
- The dot for “The puppy made a noise” is at [11, 51, 4]. (Very close!)  
- The dot for “I like pizza” is at [90, 2, 88]. (Far away!)

The Translator (Embedding Model) is the tool that takes your text and gives you those coordinates.

In [2]:
# 1. The text we want to translate
text = "The cat watched carefully as the playful kitten chased a leaf across the driveway, only stopping when a car rolled slowly past the house."


# 2. Ask Ollama to create the embedding
# We use 'nomic-embed-text' because it's built for this.
response = ollama.embeddings(
    model='nomic-embed-text',
    prompt=text
)

# 3. Get the vector (The list of numbers)
vector = response['embedding']

# 4. Inspect the result
print(f"Text: {text}")
print(f"Vector Length: {len(vector)}") # How many dimensions?
print(f"First 5 numbers: {vector[:5]}") # Just a peek

Text: The cat watched carefully as the playful kitten chased a leaf across the driveway, only stopping when a car rolled slowly past the house.
Vector Length: 768
First 5 numbers: [1.0189101696014404, 0.19626553356647491, -2.8098843097686768, 0.09355314821004868, 1.6594434976577759]


## Vector Stores

#### 1. Intuition: The Color Gradient

Imagine a library where books aren't sorted by author or title, but by **color**.

- All the **"Red"** books (Action/Adventure) are in one corner.
- All the **"Blue"** books (Sad/Melancholy) are in another.

When you walk in and say *"I want something Red-ish,"* the librarian doesn't check every book in the building.  
They walk straight to the Red corner and grab the closest matches.

A **Vector Store** is that library.  
It organizes your data so that *"math-similar"* items sit next to each other in memory.

#### 2. Visual-in-Words: The Index

- **Without Vector Store:**  
  You have a pile of 1,000 messy papers on the floor.  
  To find *"Project X,"* you pick up paper #1, read it, put it down.  
  Pick up #2, read it… *(slow)*

- **With Vector Store:**  
  You have a specialized filing cabinet where related topics are magnetized together.  
  You magnetize your query (*"Project X"*) and throw it at the cabinet.  
  It sticks immediately to the right folder.



In [3]:
import chromadb

# 1. Initialize the Client (The Librarian)
# This creates a temporary in-memory database. 
# (Use persistent_client to save to disk, but for now, memory is fine)
client = chromadb.Client()

# 2. Create a Collection (The Bookshelf)
# A 'collection' is where we store related documents.
collection = client.create_collection(name="dragon_facts") #here since we didn't add any model, so it will download a default embedding model

# 3. Add Documents (The Knowledge)
# We give each chunk of text an ID so we can find it later.
documents = [
    "My dragon's name is Fluffy.",
    "Fluffy eats 500kg of spicy tacos every Tuesday.",
    "Fluffy lives in a volcano behind the Walmart."
]
ids = ["id1", "id2", "id3"]

# 4. Store them
# NOTICE: Chroma handles the embedding part automatically in the background!
collection.add(
    documents=documents,
    ids=ids
)

print("Library built! I currently know", collection.count(), "facts about the dragon.")

Library built! I currently know 3 facts about the dragon.


## Retrival

#### 1. Intuition: The "Needle in the Haystack"

You have a massive haystack (your database).  
You throw a magnet (your query) into it.  
The magnet pulls out the 3 needles that are most magnetic (most similar).

**Retrieval is simply:** Query Vector vs. Database Vectors.

#### 2. Visual-in-Words: The Search

1. **User Query:**  
   *"What does the dragon eat?"*

2. **Translation:**  
   We convert that question into a vector (numbers).

3. **Comparison:**  
   The database compares that vector against:
   - "Fluffy's Name"
   - "Fluffy's Food"
   - "Fluffy's Home"

4. **Ranking:**
   - Match with **"Fluffy Name"**: 10% match  
   - Match with **"Fluffy Home"**: 15% match  
   - Match with **"Fluffy Food"**: 92% match  

5. **Result:**  
   It returns the **"Food"** document.


In [4]:
# 1. Setup (Same as before)
client = chromadb.Client()
collection = client.create_collection(name="dragon_facts_test")

documents = [
    "My dragon's name is Fluffy.",
    "Fluffy eats 500kg of spicy tacos every Tuesday.",
    "Fluffy lives in a volcano behind the Walmart."
]
collection.add(documents=documents, ids=["id1", "id2", "id3"])

# ---------------------------------------------------------
# 2. The Retrieval Step
# ---------------------------------------------------------

query_text = "Tell me about the creature's diet"

results = collection.query(
    query_texts=[query_text],
    n_results=1  # How many top matches do you want? Let's ask for the best ONE.
)

# 3. Inspect the results
print("Question:", query_text)
print("\n--- Retrieved Document ---")
print(results['documents'][0][0]) # It returns a list of lists, so we grab the first item.

Question: Tell me about the creature's diet

--- Retrieved Document ---
Fluffy eats 500kg of spicy tacos every Tuesday.


## The Full RAG Pipeline

1. Setup: Initialize Ollama (Brain) and Chroma (Library).
2. Ingest: Read data, Embed it (using Ollama), Store it.
3. Ask: User asks a question.
4. Retrieve: Find the best chunk.
5. Augment: Create a prompt: "Context: {chunk}. Question: {question}."
6. Generate: Get the final answer.

In [5]:
import ollama
import chromadb
from chromadb.utils import embedding_functions

# ---------------------------------------------------------
# 1. SETUP: Connect the Brain and the Library
# ---------------------------------------------------------

# We tell Chroma: "Don't use your default. Use Ollama's 'nomic-embed-text' model."
# Make sure you ran: `ollama pull nomic-embed-text` in terminal first!
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings", # Standard Ollama endpoint
)

client = chromadb.Client()
collection = client.create_collection(
    name="dragon_rag", 
    embedding_function=ollama_ef # <--- KEY CHANGE: Using Ollama for embeddings
)

# ---------------------------------------------------------
# 2. INGEST: Teach the AI
# ---------------------------------------------------------
# Pro Tip: Repeating the subject ("The dragon Fluffy") helps retrieval!
docs = [
    "The dragon's name is Fluffy.",
    "The dragon Fluffy eats 500kg of spicy tacos every Tuesday.",
    "The dragon Fluffy lives in a volcano behind the Walmart."
]
ids = ["id1", "id2", "id3"]

collection.add(documents=docs, ids=ids)

# ---------------------------------------------------------
# 3. ASK & RETRIEVE
# ---------------------------------------------------------
user_question = "Where does the beast live?" # Tricky question (Beast != Dragon)

# Look it up!
results = collection.query(
    query_texts=[user_question],
    n_results=1
)
best_chunk = results['documents'][0][0]

print(f"🔎 Retrieved Context: '{best_chunk}'")

# ---------------------------------------------------------
# 4. AUGMENT & GENERATE
# ---------------------------------------------------------
# This is the "Prompt Engineering" part.
prompt = f"""
You are an expert on mythical creatures.
Use the following context to answer the question.
If the answer is not in the context, say "I don't know".

Context: {best_chunk}

Question: {user_question}
"""

print("\n🤖 Generating Answer...")
response = ollama.chat(
    model='gemma3:4b', # Or 'llama3' or 'mistral'
    messages=[{'role': 'user', 'content': prompt}]
)

print("--------------------------------------------------")
print(response['message']['content'])
print("--------------------------------------------------")

🔎 Retrieved Context: 'The dragon Fluffy lives in a volcano behind the Walmart.'

🤖 Generating Answer...
--------------------------------------------------
The beast lives in a volcano behind the Walmart.
--------------------------------------------------


## The Elephant's Brain (Chat History)  
#### 1. Intuition: The Amnesia Patient & The Diary  
The Problem:  
If you ask: *"Who is Fluffy?"* → It answers: *"A dragon."*  
Then you ask: *"What does he eat?"* → It asks: *"Who is 'he'?"*  

The Solution (Memory):  
Since the AI has amnesia, we must keep a **Diary** (List) of everything said so far.  
Every time we send a new message, we hand the entire Diary to the AI so it can catch up on the context before answering.

#### 2. Visual-in-Words: The Stack of Plates  
Turn 1: You put a plate down (User: *"Hi"*).  
The AI puts a plate on top (AI: *"Hello"*).  

Turn 2: You put a new plate (User: *"How are you?"*).  

The **Context Window**:  
The AI looks at the entire stack of plates from bottom to top to understand the conversation flow.

In [1]:
import ollama
import chromadb
from chromadb.utils import embedding_functions

# ---------------------------------------------------------
# 1. SETUP (Same as before)
# ---------------------------------------------------------
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings"
)
client = chromadb.Client()
collection = client.create_collection(name="dragon_chat_history2", embedding_function=ollama_ef)

# Add Data
docs = [
    "The dragon Fluffy eats 500kg of spicy tacos.",
    "Fluffy lives in a volcano."
]
ids = ["id1", "id2"]
collection.add(documents=docs, ids=ids)

# ---------------------------------------------------------
# 2. INITIALIZE MEMORY
# ---------------------------------------------------------
# We start with a System message to set the behavior
conversation_history = [
    {'role': 'system', 'content': 'You are a helpful assistant. Be brief.'}
]

print("💬 Chat initialized! (Type 'exit' to stop)")

# ---------------------------------------------------------
# 3. THE CHAT LOOP
# ---------------------------------------------------------
while True:
    # A. Get User Input
    user_input = input("\nYou: ")
    if user_input.lower() == "exit":
        break

    # B. Retrieval (RAG)
    # We look for facts related to the CURRENT question
    results = collection.query(query_texts=[user_input], n_results=1)
    context_data = results['documents'][0][0]

    # C. Construct the Prompt with Context
    # We don't save the HUGE context to history (it wastes space), 
    # we just use it for this specific turn.
    prompt_with_context = f"""
    
    You are a strict assistant. You must ONLY use the provided Context info to answer.
    If the answer is not in the Context info, you must say "I don't know".
    Do not use your own external knowledge.

    Context info: {context_data}
    User Question: {user_input}
    """

    # D. Prepare Message List for the AI
    # Take the FULL history + the NEW augmented prompt
    messages_to_send = conversation_history + [{'role': 'user', 'content': prompt_with_context}]

    # E. Generate
    response = ollama.chat(
        model='gemma3:4b',
        messages=messages_to_send
    )
    ai_answer = response['message']['content']
    print(f"AI: {ai_answer}")

    # F. Update Memory (CRITICAL STEP)
    # We save the Original User Question (without the messy context) and the AI Answer
    # This keeps the history clean for the next turn.
    conversation_history.append({'role': 'user', 'content': user_input})
    conversation_history.append({'role': 'assistant', 'content': ai_answer})

💬 Chat initialized! (Type 'exit' to stop)
AI: Fluffy is the dragon.
AI: 500kg of spicy tacos.
AI: I don't know.


## The Judge (Evaluation)

You have built the car. You have driven it. You have seen it crash (hallucinate).  
Now, how do we measure how good it is without manually chatting for hours?

#### 1. Intuition: The Exam  
- **Manual Testing:** You ask 100 questions. You read 100 answers. (Takes all day).  
- **Automated Evaluation:** You have an “Answer Key”. You write a script to check if the AI’s answer matches the key.

#### 2. Visual-in-Words: RAGAS (The Scorecard)  
There is a standard framework called **RAGAS** (Retrieval-Augmented Generation Assessment). It measures:  

- **Faithfulness:** Did the AI stick to the context? (No “Mystaria”.)  
- **Answer Relevance:** Did it actually answer the question? (No “I like turtles.”)  
- **Context Precision:** Did the retrieval find the right chunk?  

Since installing RAGAS libraries can be heavy, we will build a **Simple Judge** using the LLM itself.